# Making chloropleth maps in Altair

Here's a quick example of how to make a chloropleth map in Altair.  In this example, we'll work with a fairly large data set of baby names in France from 1900-2019, broken down by department.

To work with geographical data, we'll use the `geopandas`, which loads `pandas` dataframes, but with support for geographical outlines in the `geojson` format.  You can use these dataframes just as you would a regular `pandas` dataframe, but they will include that extra geographical outline data.

To get started, we'll need to import our libraries.

In [3]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [5]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
2608753,2,INÈS,2005,14,38
2328550,2,ELISE,1901,43,5
3651388,2,TYPHAINE,1994,29,14
2006316,2,BEATRIX,1958,34,8
94215,1,ALI,2012,56,4


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [6]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
9,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ..."
26,28,Eure-et-Loir,"POLYGON ((0.81482 48.67017, 0.82767 48.68072, ..."
80,80,Somme,"POLYGON ((1.38155 50.06577, 1.45388 50.11033, ..."
71,71,Saône-et-Loire,"POLYGON ((4.11597 47.12334, 4.15377 47.11456, ..."
23,25,Doubs,"POLYGON ((6.80701 47.5628, 6.81666 47.54792, 6..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [7]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
712656,92,Hauts-de-Seine,"POLYGON ((2.29097 48.95097, 2.32697 48.94536, ...",1,ILIES,2007,92,3
117536,76,Seine-Maritime,"POLYGON ((1.38155 50.06577, 1.40926 50.05707, ...",1,ANGELO,1981,76,3
593098,56,Morbihan,"MULTIPOLYGON (((-3.42179 47.62, -3.44067 47.62...",1,GAUTIER,1993,56,6
1116954,87,Haute-Vienne,"POLYGON ((0.82343 46.12858, 0.83345 46.16655, ...",1,MATHIEU,1975,87,7
3299257,23,Creuse,"POLYGON ((2.16779 46.42407, 2.19757 46.4283, 2...",2,ODETTE,1948,23,9


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [ ]:
numeric_columns = names.select_dtypes(include=['number']).columns

grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False)[numeric_columns].sum()

grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt')

Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [7]:
name = 'LUCIEN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600)

alt.Chart(...)

## Visualisation 1

In [ ]:
df = pd.DataFrame(just_names)

grouped = df.groupby(['annais', 'preusuel'], as_index=False)['nombre'].sum()
top_names_per_year = (
    grouped.sort_values(['annais', 'nombre'], ascending=[True, False])
    .groupby('annais')
    .head(20)
)

top_names_per_year['annais'] = top_names_per_year['annais'].astype(int)

year_param = alt.param(
    name='year_param',
    bind=alt.binding_range(
        min=int(top_names_per_year['annais'].min()),
        max=int(top_names_per_year['annais'].max()),
        step=1,
        name='Année :'
    ),
    value=int(top_names_per_year['annais'].min())
)

chart = alt.Chart(top_names_per_year).add_params(
    year_param
).transform_filter(
    alt.datum.annais == year_param
).mark_bar().encode(
    x=alt.X('nombre:Q', title='Nombre de bébés'),
    y=alt.Y('preusuel:N', sort='-x', title='Prénom'),
    color=alt.Color('preusuel:N', legend=None),
    tooltip=['preusuel:N', 'nombre:Q']
).properties(
    width=1000,
    height=600,
    title='Top 10 Prénoms par Année'
)

chart.show()

alt.Chart(...)

## Visualisation 2

In [ ]:

selected_names = ['ALAIN', 'ANDRE', 'JEAN', 'JEANNE', 'LOUIS', 'MARIE', 'MICHEL', 'PHILIPPE', 'PIERRE', 'RENE']
filtered_names = names[names['preusuel'].isin(selected_names)]

numeric_columns = filtered_names.select_dtypes(include=['number']).columns

grouped = filtered_names.groupby(['dpt', 'preusuel', 'sexe', 'annais'], as_index=False)[numeric_columns].sum()

grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt')

name_dropdown = alt.binding_select(options=sorted(grouped['preusuel'].unique().tolist()))
name_select = alt.selection_point(fields=['preusuel'], bind=name_dropdown, name='Select a name:')

year_slider = alt.binding_range(min=int(grouped['annais'].min()), max=int(grouped['annais'].max()), step=1)
year_select = alt.selection_point(fields=['annais'], bind=year_slider, name='Select a year:')

chart = alt.Chart(grouped).mark_geoshape(stroke='white').encode(
    tooltip=['nom:N', 'code:N', 'nombre:Q'],
    color=alt.Color('nombre:Q')
).add_params(
    name_select,
    year_select
).transform_filter(
    name_select
).transform_filter(
    year_select
).properties(
    width=800,
    height=600,
    title='Regional Distribution of Baby Names in France'
)

chart.show()


alt.Chart(...)

### Visualisation 3 : 

In [ ]:
## find unixes names 
grouped = names.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()
pivoted = grouped.pivot(index='preusuel', columns='sexe', values='nombre').fillna(0)
pivoted.columns = ['Girls', 'Boys']  

pivoted['Total'] = pivoted['Girls'] + pivoted['Boys']
pivoted['ProportionDiff'] = abs(pivoted['Girls'] - pivoted['Boys']) / pivoted['Total']

filtered = pivoted[pivoted['Total'] > 1000]

unisex_names = filtered.sort_values('ProportionDiff').head(10)

print(unisex_names)

              Girls      Boys     Total  ProportionDiff
preusuel                                               
YAEL         1615.0    1666.0    3281.0        0.015544
CHARLIE     10060.0   10465.0   20525.0        0.019732
JANICK        774.0     870.0    1644.0        0.058394
LOUISON      3146.0    3987.0    7133.0        0.117903
JANY          668.0     900.0    1568.0        0.147959
YAËL          863.0     626.0    1489.0        0.159167
DOMINIQUE  238623.0  165887.0  404510.0        0.179813
SASHA        8380.0    5800.0   14180.0        0.181946
MAE          1234.0    2123.0    3357.0        0.264820
GABY          447.0     779.0    1226.0        0.270799


In [17]:

filtered = names[names['preusuel'].isin(['CHARLIE', 'MARIE', 'CAMILLE', 'YAEL', 'JANICK',  'LOUISON', 'JANY', 'DOMINIQUE', 'SASHA', 'MAE', 'GABY'])]  # use your list
filtered['annais'] = pd.to_numeric(filtered['annais'], errors='coerce')
filtered = filtered.dropna(subset=['annais'])
filtered['annais'] = filtered['annais'].astype(int)

grouped = filtered.groupby(['preusuel', 'sexe', 'annais'], as_index=False)['nombre'].sum()

grouped['nombre_signed'] = grouped.apply(
    lambda row: -row['nombre'] if row['sexe'] == 2 else row['nombre'], axis=1
)

grouped['sexe'] = grouped['sexe'].astype(str)

chart = alt.Chart(grouped).mark_bar().encode(
    y=alt.Y('annais:O', title='Year', sort='descending'),
    x=alt.X('nombre_signed:Q', title='Number of births', axis=alt.Axis(format='~s')),
    color=alt.Color('sexe:N',
                    scale=alt.Scale(domain=['1', '2'], range=['steelblue', 'hotpink']),
                    title='Sex',
                    legend=alt.Legend(labelExpr="datum.value == '1' ? 'Boys' : 'Girls'")),
    tooltip=['preusuel:N', 'annais:O', 'sexe:N', 'nombre:Q']
).properties(
    width=700,
    height=1000
).facet(
    row=alt.Row('preusuel:N', title=None)
).resolve_scale(
    x='independent'
)

chart.show()


/Users/nournounah/Library/Python/3.9/lib/python/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


alt.FacetChart(...)